## Working with embedding layers and 1d convolution layers 
- examples using sequential models
- demos of dropout for sequential models and bidirectional sequential layers along the way

Code adjusted from *Deep Learning for Python* Book

In [1]:
# Get raw imdb dataset
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-05-04 13:44:37--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  18.2MB/s    in 8.4s    

2020-05-04 13:44:46 (9.56 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
# Untar it to a new folder
! tar xf aclImdb_v1.tar.gz

In [0]:
# Build corpus of docs and labels
import os

imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [4]:
print(texts[0])
print(labels[0])

I am an avid B-Rate horror film buff and have viewed my fair share of slasher pictures, so I have a substantial gauge to judge this film by. It easily ranks in the upper echelon of the worst horror films the 1980's has to offer. It isn't as scary as Night of the Demons, it isn't as gory as Re-Animator and lacks the camp value of There's Nothing Out There. That being said, this film has no value. Keep in mind, the movie artwork is for a completely different film. The stills shots on the back of the DVD box aren't taken from this film.<br /><br />VIOLENCE: $$$ (There is plenty of violence but we've seen it all before. A murderer kills nubile students and the occasional facility member by slitting throats and all the other tired methods of murder that horror films utilize).<br /><br />NUDITY: None <br /><br />STORY: $$ (The story focuses on Francine Forbes - who wisely changed her name to Forbes Riley after this film was made - who accepts a job teaching at a university. People start to d

In [5]:
# Tokenize the data into one hot vectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = 200  # We will be training on 200 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) # converts words in each text to each word's numeric index in tokenizer dictionary.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples] #200 words
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Using TensorFlow backend.


Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [6]:
# Let's start with a model that ignores the sequential steps that make up each observation
from keras.layers import Dense, Embedding
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
# Specify the size of your vocabulary (i.e.-10,000 terms)
# Specify the number of features you want to extract via fitting weights to your embedding matrix.
# We also specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs 
model.add(Embedding(10000, 8, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 8)            80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 801       
Total params: 80,801
Trainable params: 80,801
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6939 - acc: 0.5000 - val_loss: 0.6884 - val_acc: 0.6250
Epoch 2/10
160/160 [==============================] - 0s 195us/step - loss: 0.6701 - acc: 0.8438 - val_loss: 0.6882 - val_acc: 0.6250
Epoch 3/10
160/160 [==============================] - 0s 113us/step - loss: 0.6526 - acc: 0.9563 - val_loss: 0.6880 - val_acc: 0.6250
Epoch 4/10
160/160 [==============================] - 0s 114us/step - loss: 0.6359 - acc: 0.9750 - val_loss: 0.6877 - val_acc: 0.6000
Epoch 5/10
160/160 [==============================] - 0s 98us/step - loss: 0.6189 - acc: 0.9812 - val_loss: 0.6874 - val_acc: 0.5750
Epoch 6/10
160/160 [==============================] - 0s 98us/step - loss: 0.6008 - acc: 0.9937 - val_loss: 0.6871 - val_acc: 0.6000
Epoch 7/10
160/160 [==============================] - 0s 103us/step - loss: 0.5820 - acc: 0.9937 - val_loss: 0.6866 - val_acc: 0.6000
Epoch 8/10
160/160 [=

In [7]:
# What if we wanted to use a matrix of pretrained embeddings?  Same as transfer learning before, but now we are importing a pretrained Embedding matrix:
# Download Glove embedding matrix weights (Might take 10 mins or so!)
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2020-05-04 13:45:07--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2020-05-04 13:45:08--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2020-05-04 13:45:08--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [8]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [0]:
# Extract embedding data for 100 feature embedding matrix
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [0]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [0]:
# Set up same model architecture as before and then import Glove weights to Embedding layer:

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()



Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_4 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [0]:

# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False



model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')


# Training data small to speed up training. Increase for better fit.


Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 1s 3ms/step - loss: 1.4497 - acc: 0.5050 - val_loss: 0.7789 - val_acc: 0.5032
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: 0.5007 - acc: 0.7650 - val_loss: 1.2360 - val_acc: 0.5042
Epoch 3/10
200/200 [==============================] - 0s 2ms/step - loss: 0.4402 - acc: 0.8350 - val_loss: 0.8863 - val_acc: 0.5026
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: 0.4775 - acc: 0.7550 - val_loss: 1.1220 - val_acc: 0.4960
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.2486 - acc: 0.9300 - val_loss: 0.8481 - val_acc: 0.5065
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.1726 - acc: 0.9950 - val_loss: 0.9987 - val_acc: 0.5024
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.1658 - acc: 0.9850 - val_loss: 0.7854 - val_acc: 0.5096
Epoch 8/10
200/200 [========

In [0]:
# Evaluate model on test set (need to preprocess test data to same structure first)

test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

#using tokenizer object we fit to test data above
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [0]:
model.load_weights('pre_trained_glove_model.h5')

model.evaluate(x_test, y_test)



25000/25000 [==============================] - 1s 44us/step


[0.8142335695266724, 0.5177599787712097]

## Fitting sequential models to text data

In [0]:
# Example 1: simple RNN
from keras.layers import SimpleRNN, LSTM
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

# Small training data.  Increase for model improvement

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6996 - acc: 0.5000 - val_loss: 0.7073 - val_acc: 0.4000
Epoch 2/10
160/160 [==============================] - 0s 726us/step - loss: 0.5300 - acc: 0.9187 - val_loss: 0.7142 - val_acc: 0.4750
Epoch 3/10
160/160 [==============================] - 0s 787us/step - loss: 0.4217 - acc: 0.9563 - val_loss: 0.7256 - val_acc: 0.4000
Epoch 4/10
160/160 [==============================] - 0s 747us/step - loss: 0.2712 - acc: 1.0000 - val_loss: 0.8405 - val_acc: 0.3500
Epoch 5/10
160/160 [==============================] - 0s 729us/step - loss: 0.1914 - acc: 1.0000 - val_loss: 0.7606 - val_acc: 0.4500
Epoch 6/10
160/160 [==============================] - 0s 707us/step - loss: 0.1621 - acc: 1.0000 - val_loss: 0.9073 - val_acc: 0.3750
Epoch 7/10
160/160 [==============================] - 0s 704us/step - loss: 0.1365 - acc: 1.0000 - val_loss: 0.8559 - val_acc: 0.3500
Epoch 8/10
160/160 

In [0]:
# Example 2: Stacked RNN layers

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

# Small training data.  Increase for model improvement

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 1s 7ms/step - loss: 0.7390 - acc: 0.4500 - val_loss: 0.7176 - val_acc: 0.5000
Epoch 2/10
160/160 [==============================] - 0s 2ms/step - loss: 0.5616 - acc: 0.7625 - val_loss: 0.8138 - val_acc: 0.5000
Epoch 3/10
160/160 [==============================] - 0s 2ms/step - loss: 0.2477 - acc: 0.9625 - val_loss: 0.9163 - val_acc: 0.3750
Epoch 4/10
160/160 [==============================] - 0s 2ms/step - loss: 0.0842 - acc: 1.0000 - val_loss: 0.9652 - val_acc: 0.4500
Epoch 5/10
160/160 [==============================] - 0s 2ms/step - loss: 0.0409 - acc: 1.0000 - val_loss: 1.0614 - val_acc: 0.4500
Epoch 6/10
160/160 [==============================] - 0s 2ms/step - loss: 0.0255 - acc: 1.0000 - val_loss: 1.0802 - val_acc: 0.4250
Epoch 7/10
160/160 [==============================] - 0s 2ms/step - loss: 0.0179 - acc: 1.0000 - val_loss: 1.1149 - val_acc: 0.4500
Epoch 8/10
160/160 [===========

In [0]:
# Example 3: LSTM layer

model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

# Small training data.  Increase for model improvement

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 1s 4ms/step - loss: 0.6920 - acc: 0.5625 - val_loss: 0.6952 - val_acc: 0.5000
Epoch 2/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6789 - acc: 0.5625 - val_loss: 0.6972 - val_acc: 0.5000
Epoch 3/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6618 - acc: 0.5625 - val_loss: 0.7067 - val_acc: 0.5000
Epoch 4/10
160/160 [==============================] - 0s 2ms/step - loss: 0.5954 - acc: 0.5750 - val_loss: 1.1216 - val_acc: 0.5000
Epoch 5/10
160/160 [==============================] - 0s 2ms/step - loss: 0.5686 - acc: 0.7563 - val_loss: 0.7647 - val_acc: 0.5000
Epoch 6/10
160/160 [==============================] - 0s 2ms/step - loss: 0.4072 - acc: 0.7875 - val_loss: 0.6794 - val_acc: 0.5000
Epoch 7/10
160/160 [==============================] - 0s 2ms/step - loss: 0.3419 - acc: 0.9500 - val_loss: 0.8986 - val_acc: 0.5250
Epoch 8/10
160/160 [===========

In [0]:
#Example 4: Bidirectional LSTM

model = Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 1s 7ms/step - loss: 0.6945 - acc: 0.4688 - val_loss: 0.6948 - val_acc: 0.4000
Epoch 2/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6870 - acc: 0.7812 - val_loss: 0.6999 - val_acc: 0.3500
Epoch 3/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6815 - acc: 0.6000 - val_loss: 0.7051 - val_acc: 0.3500
Epoch 4/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6713 - acc: 0.5375 - val_loss: 0.7139 - val_acc: 0.3500
Epoch 5/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6583 - acc: 0.5188 - val_loss: 0.7277 - val_acc: 0.3500
Epoch 6/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6358 - acc: 0.5188 - val_loss: 0.7526 - val_acc: 0.3500
Epoch 7/10
160/160 [==============================] - 0s 2ms/step - loss: 0.5859 - acc: 0.6500 - val_loss: 0.9989 - val_acc: 0.3500
Epoch 8/10
160/160 [===========

### Some Dropout Examples for LSTM layers
Dropout was a difficult puzzle for sequential models.  Solved relatively recently by dropping out on same hidden node locations at each time step.

LSTM(128, dropout=0.2, recurrent_dropout=0.2)) 

    dropout: Float between 0 and 1.  
        Fraction of the units to drop for  
        the linear transformation of the inputs.  
    recurrent_dropout: Float between 0 and 1.  
        Fraction of the units to drop for  
        the linear transformation of the recurrent state.

In [0]:


model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 1s 7ms/step - loss: 0.6916 - acc: 0.5000 - val_loss: 0.6946 - val_acc: 0.5000
Epoch 2/10
160/160 [==============================] - 1s 4ms/step - loss: 0.6820 - acc: 0.5625 - val_loss: 0.6993 - val_acc: 0.5000
Epoch 3/10
160/160 [==============================] - 1s 4ms/step - loss: 0.7992 - acc: 0.5625 - val_loss: 0.6951 - val_acc: 0.5000
Epoch 4/10
160/160 [==============================] - 1s 4ms/step - loss: 0.6529 - acc: 0.5625 - val_loss: 0.6949 - val_acc: 0.5000
Epoch 5/10
160/160 [==============================] - 1s 4ms/step - loss: 0.6334 - acc: 0.5875 - val_loss: 0.6952 - val_acc: 0.5000
Epoch 6/10
160/160 [==============================] - 1s 4ms/step - loss: 0.5920 - acc: 0.6562 - val_loss: 0.6905 - val_acc: 0.5000
Epoch 7/10
160/160 [==============================] - 1s 4ms/step - loss: 0.6054 - acc: 0.8313 - val_loss: 0.6833 - val_acc: 0.5500
Epoch 8/10
160/160 [===========

## Sequential models using 1D Convnets

In [0]:

from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
max_len = 500  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)



Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [0]:
# Use 1D Conv layer rather than RNN or LSTM or GRU to fit model
# Why? Much lighter model to fit. Here we are training on the full dataset.  If you try
# to build a model using LSTM code after running this one it will be much slower.

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu')) 
model.add(layers.MaxPooling1D(5)) #
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)



Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
___________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.7465 - acc: 0.5114 - val_loss: 0.6882 - val_acc: 0.5334
Epoch 2/10
20000/20000 [==============================] - 72s 4ms/step - loss: 0.6674 - acc: 0.6584 - val_loss: 0.6661 - val_acc: 0.6488
Epoch 3/10
20000/20000 [==============================] - 72s 4ms/step - loss: 0.6242 - acc: 0.7564 - val_loss: 0.6144 - val_acc: 0.7392
Epoch 4/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.5335 - acc: 0.8098 - val_loss: 0.4927 - val_acc: 0.7974
Epoch 5/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.4131 - acc: 0.8504 - val_loss: 0.4127 - val_acc: 0.8354
Epoch 6/10
20000/20000 [==============================] - 74s 4ms/step - loss: 0.3401 - acc: 0.8689 - val_loss: 0.3998 - val_acc: 0.8392
Epoch 7/10
20000/20000 [==============================] - 80s 4ms/step - loss: 0.2999 - acc: 0.8730 - val_loss: 0.3837 - val_acc: